In [10]:
import mysql.connector
import config
import yelp_key
import requests
import json

In [702]:
# Establish connection to database
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.passwd)

c = cnx.cursor()

In [3]:
key = yelp_key.key

## Business Search Endpoint

In [27]:
count = 0
#Business Search Endpoint
url = 'https://api.yelp.com/v3/businesses/search'

term = 'coffee'
location = "56 Bogart St, Brooklyn, NY 11206"
latitude = 40.710946
longitude = -73.951152
sort_by = 'distance'


headers = {
            'Authorization': 'Bearer {}'.format(key),
           }

url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50,
                'latitude' : 40.710946,
                'longitude' : -73.951152,
                'offset': count, 
                'radius' : 5500,
                'sort_by' : sort_by.replace(' ', '+')
          }

In [19]:
def business_search(url, headers, params):
    response = requests.get(url, headers = headers, params = url_params)
    business = json.loads(response.text)
    return business


In [28]:
business_data = business_search(url, headers = headers, params = url_params)

In [29]:
len(business_data['businesses'])

50

In [34]:
#how many cafes were returned?
business_data['total']

4800

There are 4800 cafe's total in response

In [36]:
#preview first result 
business_data['businesses'][0]

{'id': 'm-Syr8BpEbgK-6zSgY_7Kg',
 'alias': 'cafe-argentino-brooklyn',
 'name': 'Cafe Argentino',
 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/ve7_oOWMKev1hmlxYlLxZg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/cafe-argentino-brooklyn?adjust_creative=d31wiC8swpcod_WEbvT7ew&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=d31wiC8swpcod_WEbvT7ew',
 'review_count': 250,
 'categories': [{'alias': 'argentine', 'title': 'Argentine'},
  {'alias': 'steak', 'title': 'Steakhouses'},
  {'alias': 'cafes', 'title': 'Cafes'}],
 'rating': 4.0,
 'coordinates': {'latitude': 40.711199, 'longitude': -73.95137},
 'transactions': ['restaurant_reservation', 'pickup', 'delivery'],
 'price': '$$',
 'location': {'address1': '499 Grand St',
  'address2': '',
  'address3': '',
  'city': 'Brooklyn',
  'zip_code': '11211',
  'country': 'US',
  'state': 'NY',
  'display_address': ['499 Grand St', 'Brooklyn, NY 11211']},
 'phone': '+17187829477',
 'display_phone': '(71

What Attributes are of interest from this endpoint?
- id
- name
- review count
- rating
- price
- location > address1 +
- city
- zip
- state
- phone

What might be collected from the business/details?

- id
- name
- review_count
- rating
- price
- location > address 1 + city + zip + state
- lat / long geocode
- phone
- categories

(basically all the same fields as business/search... but with geocode available


business/reviews endpoint:

for each review..

- id 
- rating
- user > id, name
- text


In [75]:
len(business_data['businesses'])

50

In [220]:
def parse_business(business_json):
    
    parsed_businesses = []
    
    for biz in business_json['businesses']:
        try:
            tup = (biz['id'], biz['name'], biz['rating'],len(biz['price']),
                   ' '.join(biz['location']['display_address']),
                   biz['display_phone'], biz['review_count'],
                   biz['categories'][0]['title'])
            parsed_businesses.append(tup)
        except:
            tup = (biz['id'], biz['name'], biz['rating'], 0,
                       ' '.join(biz['location']['display_address']),
                     biz['display_phone'], biz['review_count'],
                       biz['categories'][0]['title'])

            parsed_businesses.append(tup)
            
    return parsed_businesses
            

In [221]:
biz_data = parse_business(business_data)

In [79]:
len(biz_data)

50

In [369]:
biz_data[0][0]

'm-Syr8BpEbgK-6zSgY_7Kg'

In [90]:
def get_review_url(biz_data):
    review_url = []
    for i in range(len(biz_data)):
        review_url.append(f'https://api.yelp.com/v3/businesses/{biz_data[i][0]}/reviews')
    return review_url

In [92]:
review_url = get_review_url(biz_data)

In [241]:
biz_data

[('m-Syr8BpEbgK-6zSgY_7Kg',
  'Cafe Argentino',
  4.0,
  2,
  '499 Grand St Brooklyn, NY 11211',
  '(718) 782-9477',
  250,
  'Argentine'),
 ('DO91C6baD0RSO-AN6cKCCA',
  'The West',
  4.0,
  2,
  '379 Union Ave Brooklyn, NY 11211',
  '(718) 599-1704',
  224,
  'Coffee & Tea'),
 ('csbxFFvKpNoXXXarxv7Wsg',
  'Starbucks',
  2.5,
  2,
  '405-409 Union Ave Brooklyn, NY 11211',
  '(718) 384-6936',
  83,
  'Coffee & Tea'),
 ('esrgbcohjRBpNZhDi4ErFA',
  'Campbell & Co',
  4.5,
  2,
  '502 Lorimer St Brooklyn, NY 11211',
  '(718) 387-2267',
  94,
  'Grocery'),
 ('EFhB9X55C5XYqGbWKYzZNA',
  'Kettl',
  5.0,
  2,
  '150 Ainslie St Fl 2 Brooklyn, NY 11211',
  '(917) 514-2159',
  13,
  'Coffee & Tea'),
 ('IaWV1RwCWc8w_UrIClTTdQ',
  'The Grasshopper',
  4.5,
  0,
  '561 Lorimer St New York, NY 11211',
  '(347) 889-5413',
  12,
  'Coffee & Tea'),
 ('qMivpOAPf8mrwkb3qOm_Qw',
  'Willburg Cafe',
  3.5,
  2,
  '623 Grand St Brooklyn, NY 11211',
  '(718) 963-0219',
  185,
  'Breakfast & Brunch'),
 ('hK14kg

In [100]:
def review_call(review_urls, headers):
    review_data = []
    for urls in review_urls:
        response = requests.get(urls, headers = headers)
        review = json.loads(response.text)
        review_data.append(review)
    return review_data

In [101]:
review_raw = review_call(review_url, headers)

In [311]:
review_raw[0]

{'reviews': [{'id': 'xUG_lVmFGTsqI2IafdilVg',
   'url': 'https://www.yelp.com/biz/cafe-argentino-brooklyn?adjust_creative=d31wiC8swpcod_WEbvT7ew&hrid=xUG_lVmFGTsqI2IafdilVg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=d31wiC8swpcod_WEbvT7ew',
   'text': 'It has been delicious before and it continues to provide comfort during these uncertain time. They have a rather plentiful takeout menu! From empanadas yum!...',
   'rating': 5,
   'time_created': '2020-06-21 18:43:35',
   'user': {'id': '4ecxgN6DXQSMSigQgkXIAA',
    'profile_url': 'https://www.yelp.com/user_details?userid=4ecxgN6DXQSMSigQgkXIAA',
    'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/dUJfIW3oXemLEyH6gaCZPw/o.jpg',
    'name': 'Daniela B.'}},
  {'id': 'hlaJQ7M4QMczimvtWMJwxQ',
   'url': 'https://www.yelp.com/biz/cafe-argentino-brooklyn?adjust_creative=d31wiC8swpcod_WEbvT7ew&hrid=hlaJQ7M4QMczimvtWMJwxQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=d31wiC8swpcod_WEbvT7

In [319]:
def review_parse(review_data):
    parse_list = []
    for review in review_data:
        for x in review['reviews']:
            try:
                parse_tuple = (x['id'], x['time_created'],
                               x['rating'], x['user']['id'],
                               x['user']['name'], x['text'])
                parse_list.append(parse_tuple)
            except Exception as e:
                print(e)
        
    return parse_list

In [336]:
print(len(review_raw))
print(len(biz_data))

50
50


In [320]:
parsed_reviews = review_parse(review_raw)

In [340]:
biz_and_review = []

for i in range(0,50):
    tup = (biz_data[i][0], review_raw[i]['reviews'])
    biz_and_review.append(list(tup))
    
    


In [368]:
# for i in range(len(biz_and_review)):
#     print('business ID is:  ' + biz_and_review[i][0],
#           '\n' +  'reviews:', biz_and_review[i][1][0]['id'])


TypeError: string indices must be integers

In [348]:
biz_and_review[0]

['m-Syr8BpEbgK-6zSgY_7Kg',
 [{'id': 'xUG_lVmFGTsqI2IafdilVg',
   'url': 'https://www.yelp.com/biz/cafe-argentino-brooklyn?adjust_creative=d31wiC8swpcod_WEbvT7ew&hrid=xUG_lVmFGTsqI2IafdilVg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=d31wiC8swpcod_WEbvT7ew',
   'text': 'It has been delicious before and it continues to provide comfort during these uncertain time. They have a rather plentiful takeout menu! From empanadas yum!...',
   'rating': 5,
   'time_created': '2020-06-21 18:43:35',
   'user': {'id': '4ecxgN6DXQSMSigQgkXIAA',
    'profile_url': 'https://www.yelp.com/user_details?userid=4ecxgN6DXQSMSigQgkXIAA',
    'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/dUJfIW3oXemLEyH6gaCZPw/o.jpg',
    'name': 'Daniela B.'}},
  {'id': 'hlaJQ7M4QMczimvtWMJwxQ',
   'url': 'https://www.yelp.com/biz/cafe-argentino-brooklyn?adjust_creative=d31wiC8swpcod_WEbvT7ew&hrid=hlaJQ7M4QMczimvtWMJwxQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=d31w

In [206]:
# c.execute('''CREATE DATABASE coffee IF NOT EXISTS''')

In [650]:
c.execute('''USE coffee''')

In [646]:
c.execute('''DROP TABLE cafes;
            DROP TABLE reviews;''')

In [651]:
c.execute('''CREATE TABLE cafes(

                ID VARCHAR(100) PRIMARY KEY,
                NAME VARCHAR(100),
                RATING FLOAT,
                PRICE INT,
                ADDRESS VARCHAR(160),
                PHONE VARCHAR(100),
                REVIEW_COUNT VARCHAR(100),
                CATEGORIES VARCHAR(200));'''
         )

In [ ]:
'''CREATE TABLE Orders (
    OrderID int NOT NULL,
    OrderNumber int NOT NULL,
    PersonID int,
    PRIMARY KEY (OrderID),
    CONSTRAINT FK_PersonOrder FOREIGN KEY (PersonID)
    REFERENCES Persons(PersonID)
);'''

In [706]:
c.execute(                     
            '''CREATE TABLE reviews(
            
                CAFE_ID VARCHAR(100),
                REVIEW_ID VARCHAR(100) PRIMARY KEY,
                DATE DATETIME,
                RATING FLOAT,
                USER_ID VARCHAR(100),
                NAME VARCHAR(100),
                REVIEW VARCHAR(1000)
                FOREIGN KEY (CAFE_ID)
                                   );'''
)

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'FOREIGN KEY (CAFE_ID)
                                   )' at line 10

In [ ]:
CREATE TABLE production.products (
	product_id INT PRIMARY KEY,
	product_name VARCHAR (255) NOT NULL,
	brand_id INT NOT NULL,
	category_id INT NOT NULL,
	model_year SMALLINT NOT NULL,
	list_price DECIMAL (10, 2) NOT NULL,
	FOREIGN KEY (category_id) REFERENCES production.categories (category_id) ON DELETE CASCADE ON UPDATE CASCADE,
	FOREIGN KEY (brand_id) REFERENCES production.brands (brand_id) ON DELETE CASCADE ON UPDATE CASCADE



TABLES['salaries'] = (
    "CREATE TABLE salaries ("
    "  emp_no int(11) NOT NULL,"
    "  salary int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT salaries_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

In [214]:
def cafes_insert(c, cnx, cafe_details):
    
    c.executemany('''INSERT INTO coffee.cafes
                    (ID, NAME, RATING, PRICE, ADDRESS, PHONE, REVIEW_COUNT, CATEGORIES)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s);''', cafe_details)
    cnx.commit()

In [213]:
def review_insert(c, cnx, reviews):
    
    c.executemany('''INSERT INTO coffee.reviews (CAFE_ID, DATE, RATING, USER_ID, NAME, REVIEW)
                    VALUES (%s, %s, %s, %s, %s, %s)''', reviews)
    cnx.commit()

In [222]:
cafes_insert(c, cnx, biz_data)

In [235]:
review_insert(c, cnx, parsed_reviews)

In [ ]:
while count < 4800:
    url_params['offset'] = count
    business = business_search(url, headers, params)
    parsed_business = parse_business(business)
    cafe_insert(c, cnx, parsed_business)
    review_urls = get_review_url(parsed_business)
    review_raw = review_call(review_urls)
    parsed_reviews = review_parse(review_raw)
    review_insert(c, cnx, parsed_reviews)
    count += 50
    if count > 4800:
        break

In [219]:
# c.execute('''TRUNCATE TABLE cafes''')

In [433]:
biz_data[2]

('csbxFFvKpNoXXXarxv7Wsg',
 'Starbucks',
 2.5,
 2,
 '405-409 Union Ave Brooklyn, NY 11211',
 '(718) 384-6936',
 83,
 'Coffee & Tea')

In [ ]:
def get_review_url(biz_data):
    review_url = []
    for i in range(len(biz_data)):
        review_url.append(f'https://api.yelp.com/v3/businesses/{biz_data[i][0]}/reviews')
    return review_url

In [ ]:
def review_call(biz_data, headers):
    review_url = []
    for i in range(len(biz_data)):
        review_url.append(f'https://api.yelp.com/v3/businesses/{biz_data[i][0]}/reviews')
    review_data = []
    for urls in review_url:
        response = requests.get(urls, headers = headers)
        review = json.loads(response.text)
        review_data.append(review)
    return review_data



In [414]:
dlist[0]

[{'business_id': 'm-Syr8BpEbgK-6zSgY_7Kg',
  'review_id': 'xUG_lVmFGTsqI2IafdilVg',
  'review_text': 'It has been delicious before and it continues to provide comfort during these uncertain time. They have a rather plentiful takeout menu! From empanadas yum!...'}]

dictionary construction method works to add business id to reviews for one review. Now lets try multiple reviews


In [423]:
biz_ids = []

for x in biz_data:
    biz_ids.append(x[0])

In [486]:
id_plus_reviews = []

for i in range(len(review_raw)):
    join = (biz_ids[i], review_raw[i])
    id_plus_reviews.append(list(join))
    

In [487]:
len(id_plus_reviews)

50

In [488]:
tester = id_plus_reviews[:5]

In [537]:
new_dict = list(map(lambda x: {'business_id': x[0], 'reviews' : x[1]['reviews']}, tester))

In [571]:
new_dict[0]

{'business_id': 'm-Syr8BpEbgK-6zSgY_7Kg',
 'reviews': [{'id': 'xUG_lVmFGTsqI2IafdilVg',
   'url': 'https://www.yelp.com/biz/cafe-argentino-brooklyn?adjust_creative=d31wiC8swpcod_WEbvT7ew&hrid=xUG_lVmFGTsqI2IafdilVg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=d31wiC8swpcod_WEbvT7ew',
   'text': 'It has been delicious before and it continues to provide comfort during these uncertain time. They have a rather plentiful takeout menu! From empanadas yum!...',
   'rating': 5,
   'time_created': '2020-06-21 18:43:35',
   'user': {'id': '4ecxgN6DXQSMSigQgkXIAA',
    'profile_url': 'https://www.yelp.com/user_details?userid=4ecxgN6DXQSMSigQgkXIAA',
    'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/dUJfIW3oXemLEyH6gaCZPw/o.jpg',
    'name': 'Daniela B.'}},
  {'id': 'hlaJQ7M4QMczimvtWMJwxQ',
   'url': 'https://www.yelp.com/biz/cafe-argentino-brooklyn?adjust_creative=d31wiC8swpcod_WEbvT7ew&hrid=hlaJQ7M4QMczimvtWMJwxQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_busine

In [ ]:
#check for missing reviews
for x in n_reviews:
    print(len(x['reviews']))

In [676]:
# def parse_reviews(review_dict):
#     parsed_reviews = [] 
#     for x in review_dict:
#         try:
#             if len(x['reviews']) == 3:
#                 tup = (x['business_id'], x['reviews'][0]['id'], x['reviews'][0]['time_created'],
#                         x['reviews'][0]['rating'],x['reviews'][0]['user']['id'],
#                         x['reviews'][0]['user']['name'], x['reviews'][0]['text'], 

#                         x['business_id'], x['reviews'][1]['id'], x['reviews'][1]['time_created'],
#                         x['reviews'][1]['rating'], x['reviews'][1]['user']['id'],
#                         x['reviews'][1]['user']['name'], x['reviews'][1]['text'],

#                         x['business_id'], x['reviews'][2]['id'], x['reviews'][2]['time_created'],
#                         x['reviews'][2]['rating'], x['reviews'][2]['user']['id'],
#                         x['reviews'][2]['user']['name'], x['reviews'][2]['text'])
#             elif len(x['reviews']) == 2:
#                 tup = (x['business_id'], x['reviews'][0]['id'], x['reviews'][0]['time_created'],
#                         x['reviews'][0]['rating'],x['reviews'][0]['user']['id'],
#                         x['reviews'][0]['user']['name'], x['reviews'][0]['text'], 

#                         x['business_id'], x['reviews'][1]['id'], x['reviews'][1]['time_created'],
#                         x['reviews'][1]['rating'], x['reviews'][1]['user']['id'],
#                         x['reviews'][1]['user']['name'], x['reviews'][1]['text'])
#             else:
#                 tup = (x['business_id'], x['reviews'][0]['id'], x['reviews'][0]['time_created'],
#                         x['reviews'][0]['rating'],x['reviews'][0]['user']['id'],
#                         x['reviews'][0]['user']['name'], x['reviews'][0]['text'])          
                   
                   
#             parsed_reviews.append(tup)
#         except Exception as e:
#             print(e)
#     return parsed_reviews

In [612]:
biz_data

[('m-Syr8BpEbgK-6zSgY_7Kg',
  'Cafe Argentino',
  4.0,
  2,
  '499 Grand St Brooklyn, NY 11211',
  '(718) 782-9477',
  250,
  'Argentine'),
 ('DO91C6baD0RSO-AN6cKCCA',
  'The West',
  4.0,
  2,
  '379 Union Ave Brooklyn, NY 11211',
  '(718) 599-1704',
  224,
  'Coffee & Tea'),
 ('csbxFFvKpNoXXXarxv7Wsg',
  'Starbucks',
  2.5,
  2,
  '405-409 Union Ave Brooklyn, NY 11211',
  '(718) 384-6936',
  83,
  'Coffee & Tea'),
 ('esrgbcohjRBpNZhDi4ErFA',
  'Campbell & Co',
  4.5,
  2,
  '502 Lorimer St Brooklyn, NY 11211',
  '(718) 387-2267',
  94,
  'Grocery'),
 ('EFhB9X55C5XYqGbWKYzZNA',
  'Kettl',
  5.0,
  2,
  '150 Ainslie St Fl 2 Brooklyn, NY 11211',
  '(917) 514-2159',
  13,
  'Coffee & Tea'),
 ('IaWV1RwCWc8w_UrIClTTdQ',
  'The Grasshopper',
  4.5,
  0,
  '561 Lorimer St New York, NY 11211',
  '(347) 889-5413',
  12,
  'Coffee & Tea'),
 ('qMivpOAPf8mrwkb3qOm_Qw',
  'Willburg Cafe',
  3.5,
  2,
  '623 Grand St Brooklyn, NY 11211',
  '(718) 963-0219',
  185,
  'Breakfast & Brunch'),
 ('hK14kg

In [700]:
def review_call(business_data, headers):
    review_url = []
    for i in range(len(biz_data)):
        review_url.append(f'https://api.yelp.com/v3/businesses/{business_data[i][0]}/reviews')
    review_data = []
    for urls in review_url:
        response = requests.get(urls, headers = headers)
        review = json.loads(response.text)
        review_data.append(review)
    return review_data

In [614]:
n_reviews = review_call(biz_data, headers)

In [622]:
def get_business_ids(business_data):
    ids = []
    for i in range(len(business_data)):
        ids.append(business_data[i][0])
    return ids
        
        

In [699]:
def add_business_ids(business_ids, review_data):
    id_plus_reviews = []
    for i in range(len(review_data)):
        join = (business_ids[i], review_data[i])
        id_plus_reviews.append(list(join))
        
    join_dict = list(map(lambda x: {'business_id': x[0], 'reviews' : x[1]['reviews']}, id_plus_reviews))
    return join_dict   
        


In [623]:
b_ids = get_business_ids(biz_data)
joined_review_business = add_business_id(b_ids, n_reviews)


In [698]:
def parse_reviews(review_dict):
    parsed_reviews = []
    for x in review_dict:
        for i in range(len(x['reviews'])):
            try:
                tup = (x['business_id'], x['reviews'][i]['id'], x['reviews'][i]['time_created'],
                        x['reviews'][i]['rating'],x['reviews'][i]['user']['id'],
                        x['reviews'][i]['user']['name'], x['reviews'][i]['text']) 
                parsed_reviews.append(tup)
                i+=1
            except Exception as e:
                print(e)
    return parsed_reviews

In [638]:
parsed_reviews = parse_reviews(joined_review_business)

In [639]:
len(parsed_reviews)

50

In [661]:
joined_review_business

[{'business_id': 'm-Syr8BpEbgK-6zSgY_7Kg',
  'reviews': [{'id': 'xUG_lVmFGTsqI2IafdilVg',
    'url': 'https://www.yelp.com/biz/cafe-argentino-brooklyn?adjust_creative=d31wiC8swpcod_WEbvT7ew&hrid=xUG_lVmFGTsqI2IafdilVg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=d31wiC8swpcod_WEbvT7ew',
    'text': 'It has been delicious before and it continues to provide comfort during these uncertain time. They have a rather plentiful takeout menu! From empanadas yum!...',
    'rating': 5,
    'time_created': '2020-06-21 18:43:35',
    'user': {'id': '4ecxgN6DXQSMSigQgkXIAA',
     'profile_url': 'https://www.yelp.com/user_details?userid=4ecxgN6DXQSMSigQgkXIAA',
     'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/dUJfIW3oXemLEyH6gaCZPw/o.jpg',
     'name': 'Daniela B.'}},
   {'id': 'hlaJQ7M4QMczimvtWMJwxQ',
    'url': 'https://www.yelp.com/biz/cafe-argentino-brooklyn?adjust_creative=d31wiC8swpcod_WEbvT7ew&hrid=hlaJQ7M4QMczimvtWMJwxQ&utm_campaign=yelp_api_v3&utm_medium=a